In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [25]:
from src.inference import get_feature_store

In [26]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
print(ts_data, "test")
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 10:57:02,708 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 10:57:02,747 INFO: Initializing external client
2025-03-06 10:57:02,751 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 10:57:04,518 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215695
Fetching data from 2025-02-05 15:57:02.706606+00:00 to 2025-03-06 14:57:02.706606+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 
                  pickup_hour  pickup_location_id  rides
0   2025-02-13 23:00:00+00:00                 263     50
1   2025-03-04 11:00:00+00:00                 263     98
2   2025-02-11 22:00:00+00:00                 263     57
3   2025-02-26 08:00:00+00:00                 263    145
4   2025-02-21 14:00:00+00:00                 263    127
..                        ...                 ...    ...
667 2025-02-07 23:00:00+00:00                 263    120


In [27]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 17:00:00+00:00,263,159
1,2025-02-05 18:00:00+00:00,263,129
2,2025-02-05 19:00:00+00:00,263,112
3,2025-02-05 20:00:00+00:00,263,112
4,2025-02-05 21:00:00+00:00,263,112
...,...,...,...
667,2025-03-05 12:00:00+00:00,263,100
668,2025-03-05 13:00:00+00:00,263,108
669,2025-03-05 14:00:00+00:00,263,136
670,2025-03-05 15:00:00+00:00,263,153


In [28]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_hour         672 non-null    datetime64[us, Etc/UTC]
 1   pickup_location_id  672 non-null    int32                  
 2   rides               672 non-null    int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 10.6 KB


In [29]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [30]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         672 non-null    datetime64[us]
 1   pickup_location_id  672 non-null    int32         
 2   rides               672 non-null    int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 10.6 KB


In [32]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=12*28, step_size=23)

In [34]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 10:58:55,413 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 10:58:55,429 INFO: Initializing external client
2025-03-06 10:58:55,432 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 10:58:57,160 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215695
Fetching data from 2025-02-05 15:58:55.413255+00:00 to 2025-03-06 14:58:55.413255+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


In [35]:
current_date

Timestamp('2025-03-06 15:58:55.413255+0000', tz='Etc/UTC')

In [36]:
features

,rides_t-336,rides_t-335,rides_t-334,rides_t-333,rides_t-332,rides_t-331,rides_t-330,rides_t-329,rides_t-328,rides_t-327,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,159,129,112,112,112,63,37,14,11,3,...,125,109,97,115,109,132,107,100,263,2025-02-19 17:00:00
1,103,147,120,100,114,124,96,49,44,26,...,127,123,127,98,113,135,94,110,263,2025-02-20 16:00:00
2,116,137,132,177,207,133,129,129,120,73,...,88,97,85,109,104,115,126,127,263,2025-02-21 15:00:00
3,145,126,133,161,194,180,112,109,128,101,...,17,35,53,101,134,130,152,143,263,2025-02-22 14:00:00
4,144,125,96,145,218,145,68,91,101,71,...,14,31,23,56,77,110,130,134,263,2025-02-23 13:00:00
5,116,90,114,134,111,137,105,80,79,76,...,7,15,62,121,154,119,91,92,263,2025-02-24 12:00:00
6,62,72,60,74,80,75,78,91,84,64,...,7,5,16,48,123,147,142,88,263,2025-02-25 11:00:00
7,111,108,106,118,114,145,133,150,201,176,...,5,4,7,27,55,134,145,139,263,2025-02-26 10:00:00
8,147,127,117,121,123,119,132,132,162,146,...,17,2,5,8,27,65,164,173,263,2025-02-27 09:00:00
9,125,131,119,121,123,118,133,116,111,137,...,39,41,20,8,10,31,57,146,263,2025-02-28 08:00:00


In [37]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 10:59:18,213 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 10:59:18,226 INFO: Initializing external client
2025-03-06 10:59:18,228 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 10:59:19,683 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215695


ValueError: max() arg is an empty sequence

In [18]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

NameError: name 'model' is not defined

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,5,0.0
4,6,0.0
...,...,...
251,259,0.0
252,260,1.0
253,261,10.0
254,262,10.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 249, 230, 161,  79, 142,  48, 114, 138, 163])